# 介绍
> 微软在 2020 年初开源的新一代自动化测试工具，它的功能类似于 Selenium、Pyppeteer 等，都可以驱动浏览器进行各种自动化操作
# 优点
- Playwright 的安装和配置非常简单，安装过程中会自动安装对应的浏览器和驱动，不需要额外配置 WebDriver 等。
# 局限
- Playwright不支持旧版Microsoft Edge或IE11。支持新的Microsoft Edge（在Chromium上）；所以对浏览器版本有硬性要求的项目不适用。
- 需要SSL证书进行访问的网站可能无法录制，该过程需要单独定位编写。
- 移动端测试是通过桌面浏览器来模拟移动设备（相当于自带模拟器），无法控制真机。
# 安装
执行以下两句命令即可,无需其他操作
``` bash 
pip3 install playwright
playwright install

```
# 转载博客
https://segmentfault.com/a/1190000039150175   
https://cuiqingcai.com/36045.html

# 基本使用
## 同步

In [ ]:
from playwright.sync_api import sync_playwright

# 浏览器上下文管理器 p 
with sync_playwright() as p:
    #  Chromium、Firefox 以及 Webkit 浏览器实例
    for browser_type in [p.chromium, p.firefox, p.webkit]:
        # launch 方法返回的是一个 Browser 对象,打开浏览器
        browser = browser_type.launch(headless=False)
        # new_page 方法，相当于新建了一个选项卡
        page = browser.new_page()
        # 调用 goto，就是加载某个页面
        page.goto('https://www.baidu.com')
        # 截图
        page.screenshot(path=f'screenshot-{browser_type.name}.png')
        print(page.title())
        browser.close()

In [2]:
# 在jupyter notebook中无法执行
!python script1.py

百度一下，你就知道
百度一下，你就知道
百度一下，你就知道


## 异步

In [ ]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        for browser_type in [p.chromium, p.firefox, p.webkit]:
            # 注意：如果不设置为 False，默认是无头模式启动浏览器，我们看不到任何窗口。
            browser = await browser_type.launch()
            page = await browser.new_page()
            await page.goto('https://www.baidu.com')
            await page.screenshot(path=f'screenshot-{browser_type.name}.png')
            print(await page.title())
            await browser.close()
asyncio.run(main())

In [3]:
# 在jupyter notebook中无法执行
!python script2.py

# 代码生成(操作录制)
- 查看文档
``` bash 
playwright codegen --help
```
- 例子：
``` bash 
playwright codegen -o script.py -b chromium
```
-  -o 代表输出的代码文件的名称；
- —target 代表使用的语言，默认是 python（同步模式）的操作代码，如果传入 python-async 就会生成异步模式的代码；
- -b 代表的是使用的浏览器，默认是 Chromium
-  —device 可以模拟使用手机浏览器，比如 iPhone 11，
- —lang 代表设置浏览器的语言，
- —timeout 可以设置页面加载超时时间。


In [4]:
! playwright codegen -o script.py -b firefox

# 移动端浏览器

In [ ]:
from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    # 传入的是手机的型号，比如 iPhone 12 Pro Max
    iphone_12_pro_max = p.devices['iPhone 12 Pro Max']
    # launch 方法返回的是一个 Browser 对象,打开浏览器
    browser = p.webkit.launch(headless=False)
    # 建了一个移动端 BrowserContext 对象，
    # 通过 geolocation 参数传入了经纬度信息，
    # 通过 permissions 参数传入了赋予的权限信息，
    # 最后将得到的 BrowserContext 对象赋值为 context 变量。
    context = browser.new_context(
        **iphone_12_pro_max,
        locale='zh-CN',
        geolocation={'longitude': 116.39014, 'latitude': 39.913904},
        permissions=['geolocation']
    )
    # 调用 new_page 方法创建一个新的选项卡
    page = context.new_page()
    # 然后跳转到高德地图
    page.goto('https://amap.com')
    # 并调用了 wait_for_load_state 方法等待页面某个状态完成
    # 这里我们传入的 state 是 networkidle，也就是网络空闲状态。
    # 因为在页面初始化和加载过程中，肯定是伴随有网络请求的
    page.wait_for_load_state(state='networkidle')
    # 调用 screenshot 方法获取当前页面截图
    page.screenshot(path='location-iphone.png')
    browser.close()

In [6]:
!python script3.py

# 定位
- CSS 选择器
- XPath
- Playwright 文本内容筛选，根据节点层级结构筛选等等。
- 文档：https://playwright.dev/docs/selectors

## 文本

In [7]:
# Text 选择文本是 Log in 的节点，并点击
page.click("text=Log in")

NameError: name 'page' is not defined

## CSS 选择器 + 文本

In [ ]:
# CSS+Text has-text 代表包含指定的字符串
page.click("article:has-text('Playwright')")
# # CSS+Text 代表字符串完全匹配
page.click("#nav-bar :text('Contact us')")

## CSS 选择器 + 节点关系

In [ ]:
# 选择 class 为 item-description 的节点，且该节点还要包含 class 为 item-promo-banner 的子节点。
page.click(".item-description:has(.item-promo-banner)")
# 这里选择的就是一个 input 节点，并且该 input 节点要位于文本值为 Username 的节点的右侧。
page.click("input:right-of(:text('Username'))")

# 事件触发和监听
- 官方文档：https://playwright.dev/docs/api/class-page
## on 时间监听

In [ ]:
from playwright.sync_api import sync_playwright

def on_response(response):
    if '/api/movie/' in response.url and response.status == 200:
        print(response.json())

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()
    # 监听网络响应
    page.on('response', on_response)
    page.goto('https://spa6.scrape.center/')
    page.wait_for_load_state('networkidle')
    browser.close()

In [9]:
! python script4.py

{'count': 101, 'results': [{'id': 1, 'name': '霸王别姬', 'alias': 'Farewell My Concubine', 'cover': 'https://p0.meituan.net/movie/ce4da3e03e655b5b88ed31b5cd7896cf62472.jpg@464w_644h_1e_1c', 'categories': ['剧情', '爱情'], 'published_at': '1993-07-26', 'minute': 171, 'score': 9.5, 'regions': ['中国内地', '中国香港']}, {'id': 2, 'name': '这个杀手不太冷', 'alias': 'Léon', 'cover': 'https://p1.meituan.net/movie/6bea9af4524dfbd0b668eaa7e187c3df767253.jpg@464w_644h_1e_1c', 'categories': ['剧情', '动作', '犯罪'], 'published_at': '1994-09-14', 'minute': 110, 'score': 9.5, 'regions': ['法国']}, {'id': 3, 'name': '肖申克的救赎', 'alias': 'The Shawshank Redemption', 'cover': 'https://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@464w_644h_1e_1c', 'categories': ['剧情', '犯罪'], 'published_at': '1994-09-10', 'minute': 142, 'score': 9.5, 'regions': ['美国']}, {'id': 4, 'name': '泰坦尼克号', 'alias': 'Titanic', 'cover': 'https://p1.meituan.net/movie/b607fba7513e7f15eab170aac1e1400d878112.jpg@464w_644h_1e_1c', 'categories': ['剧情'

# 元素常用操作

- 下拉选择框：selectOpion、value、labei、index
- 文件上传：setInputFiles、单个文件、多个文件、拖放上传
- 鼠标点击：click、dbclick
- 鼠标拖动：down、up
- 鼠标移动：move
- 触摸屏幕：tag
- 键盘按键：press
- 截屏、录屏：screenshot、recordVideo

# 网络劫持(可用来代码注入，数据爬取)
利用 route 方法，我们可以实现一些网络劫持和修改操作，比如修改 request 的属性，修改 response 响应结果等

In [ ]:
from playwright.sync_api import sync_playwright
import re

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()

    def cancel_request(route, request):
        route.abort()
    # 利用 route 方法，我们可以实现一些网络劫持和修改操作，比如修改 request 的属性，修改 response 响应结果等
    # 
    page.route(re.compile(r"(\.png)|(\.jpg)"), cancel_request)
    page.goto("https://spa6.scrape.center/")
    page.wait_for_load_state('networkidle')
    page.screenshot(path='no_picture.png')
    browser.close()

# 防屏蔽

In [ ]:
js="""
Object.defineProperties(navigator, {webdriver:{get:()=>undefined}});
"""
page.add_init_script(js)